In [ ]:
import pandas as pd
import numpy as np

In [ ]:
INPUT_FILE = "office_contacts_wide.csv"   # anonymized example
df = pd.read_csv(INPUT_FILE)

print("Shape:", df.shape)
df.head()

In [ ]:
df = df.copy()

# Remove columns where all values are NaN
df = df.dropna(axis=1, how='all')

# Strip whitespace from all string columns
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df.head()

In [ ]:
# Some data may contain repeated rows for the same company/contact
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]

print(f"Removed {before - after} duplicate rows.")

In [ ]:
# Example: choose relevant fields
columns_of_interest = [
    "company_name",
    "url",
    "c1_Name", "c1_Email",
    "c2_Name", "c2_Email",
    "c3_Name", "c3_Email"
]

available_cols = [col for col in columns_of_interest if col in df.columns]
clean_df = df[available_cols].copy()

clean_df.head()

In [ ]:
long_records = []

for _, row in clean_df.iterrows():
    for i in range(1, 6):  # up to 5 contacts per company
        name_col = f"c{i}_Name"
        mail_col = f"c{i}_Email"

        if name_col in clean_df and mail_col in clean_df:
            name = row.get(name_col)
            email = row.get(mail_col)

            if isinstance(email, str) and "@" in email:
                long_records.append({
                    "company_name": row.get("company_name"),
                    "contact_name": name,
                    "contact_email": email,
                    "url": row.get("url")
                })

long_df = pd.DataFrame(long_records)
long_df.head()

In [ ]:
def is_valid_email(x):
    if not isinstance(x, str):
        return False
    if "@" not in x:
        return False
    if x.lower().startswith("info@"):
        return False    # example rule: avoid generic inboxes
    return True

long_df = long_df[long_df["contact_email"].apply(is_valid_email)].reset_index(drop=True)
len(long_df)

In [ ]:
long_df["contact_email"] = long_df["contact_email"].str.lower().str.strip()
long_df.head()

In [ ]:
OUTPUT_FILE = "clean_mailing_list.csv"
long_df.to_csv(OUTPUT_FILE, index=False)

OUTPUT_FILE

In [ ]:
print("Final rows:", len(long_df))
print("Columns:", list(long_df.columns))